In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import get_mlp_activations
from hook_utils import save_activation
import haystack_utils
import hook_utils
import plotting_utils
import pythia_160m_utils

%reload_ext autoreload
%autoreload 2

In [2]:
model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)


Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


In [3]:
with open("./data/freelaw.txt", "r") as f:
    freelaw = f.read()


In [8]:
freelaw[:1000]

'     The summaries of the Colorado Court of Appeals published opinions\n  constitute no part of the opinion of the division but have been prepared by\n  the division for the convenience of the reader. The summaries may not be\n    cited or relied upon as they are not the official language of the division.\n  Any discrepancy between the language in the summary and in the opinion\n           should be resolved in favor of the language in the opinion.\n\n\n                                                                  SUMMARY\n                                                            February 8, 2018\n\n                                2018COA12\n\nNo. 14CA0144, People v. Trujillo — Criminal Law — Sentencing\n— Probation — Indeterminate Sentence\n\n     A division of the court of appeals considers whether a\n\nColorado statute authorizes imposition of a sentence to an\n\nindeterminate term of probation and whether the defendant was\n\nentitled to the benefit of amendments to the stat

In [13]:
freelaw_data = freelaw.split("\n\n\n")
len(freelaw_data)

2115

In [33]:
from collections import Counter
from nltk.util import ngrams

all_ngrams = {n:[] for n in range(10, 3, -1)}

for sentence in tqdm(freelaw_data):
    tokens = model.to_str_tokens(sentence)
    for n in range(10, 3, -1):
        x_grams = ngrams(tokens, n)
        all_ngrams[n].extend(x_grams)

  0%|          | 0/2115 [00:00<?, ?it/s]

In [34]:
common_phrases = Counter(all_ngrams[4]).most_common(20)
for phrase in common_phrases:
    print(phrase)

((' U', '.', 'S', '.'), 907)
((' F', '.', '3', 'd'), 395)
((' F', '.', '2', 'd'), 388)
(('S', '.', 'C', '.'), 341)
(('.', 'S', '.', 'C'), 339)
((' S', '.', 'Ct', '.'), 274)
(('.', 'C', '.', ' §'), 210)
((' L', '.', 'Ed', '.'), 210)
(('.', 'Ed', '.', '2'), 182)
(('Ed', '.', '2', 'd'), 182)
((' S', '.', 'W', '.'), 175)
((' United', ' States', ' v', '.'), 146)
(('.', 'W', '.', '2'), 128)
((' (', '5', 'th', ' Cir'), 127)
(('.', '3', 'd', ' at'), 126)
(('W', '.', '2', 'd'), 126)
(('.', 'S', '.', ' at'), 123)
(('5', 'th', ' Cir', '.'), 121)
(('.', '2', 'd', ' at'), 107)
((' v', '.', ' United', ' States'), 100)


In [35]:
prompt = ' 28 U.S.C.'
print(model.to_str_tokens(model.to_tokens(prompt)))

['<|endoftext|>', ' 28', ' U', '.', 'S', '.', 'C', '.']


In [36]:
model(prompt, return_type="loss", loss_per_token=True)

tensor([[1.2944e+01, 4.1906e+00, 2.8218e-03, 2.1607e-01, 3.9450e-04, 1.6760e+00,
         1.8215e-03]], device='cuda:0')